<a href="https://colab.research.google.com/github/Baheback/RepositorioDelMal/blob/main/ProyectoFinal_Crypto_analisys_gradio_autocorreccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance pandas numpy matplotlib ta gradio
from google.colab import drive
drive.mount('/content/drive')


import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
import os
from ta.momentum import RSIIndicator
from ta.trend import MACD

# Carpeta donde guardar CSVs
ruta_drive = "/content/drive/MyDrive/cripto_monitor/"
os.makedirs(ruta_drive, exist_ok=True)

# Lista de criptomonedas
criptos = {
    "BTC-USD": "Bitcoin",
    "ETH-USD": "Ethereum",
    "BNB-USD": "BNB",
    "XRP-USD": "Ripple",
    "ADA-USD": "Cardano",
    "DOGE-USD": "Dogecoin",
    "SOL-USD": "Solana",
    "DOT-USD": "Polkadot",
    "MATIC-USD": "Polygon",
    "LTC-USD": "Litecoin"
}

# Parámetros RSI (global para permitir ajustes)
rsi_buy = 30
rsi_sell = 70

def analizar_gradio(ticker):
    global rsi_buy, rsi_sell

    try:
        nombre = criptos.get(ticker, ticker)

        # Descargar datos
        data = yf.download(ticker, period="30d", interval="1d", auto_adjust=True, progress=False)
        if data.empty:
            return "❌ No se pudieron obtener datos.", None

        # Asegurar columnas simples
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        if "Close" not in data.columns:
            return "❌ Error: no hay columna 'Close'.", None

        # Series necesarias
        close_prices = data["Close"].astype(float).squeeze()

        if len(close_prices) < 15:
            return "⚠️ No hay suficientes datos para análisis técnico.", None

        # Indicadores
        rsi = RSIIndicator(close=close_prices, window=14).rsi()
        macd = MACD(close=close_prices)
        data["RSI"] = rsi
        data["MACD"] = macd.macd()
        data["Signal"] = macd.macd_signal()
        data["Tendencia"] = np.where(data["MACD"] > data["Signal"], "Alcista", "Bajista")

        # Última fila
        ultimo = data.iloc[-1]
        precio_actual = float(ultimo["Close"])
        rsi_actual = float(ultimo["RSI"])
        tendencia_actual = ultimo["Tendencia"]

        # Recomendación
        if rsi_actual < rsi_buy and tendencia_actual == "Alcista":
            decision = "Comprar"
        elif rsi_actual > rsi_sell and tendencia_actual == "Bajista":
            decision = "Vender"
        else:
            decision = "Mantener"

        # Guardar histórico CSV
        ruta_csv = f"{ruta_drive}{ticker}_historial.csv"
        data.to_csv(ruta_csv)

        # Guardar predicción
        ruta_pred = f"{ruta_drive}{ticker}_predicciones.csv"
        if os.path.exists(ruta_pred):
            df_pred = pd.read_csv(ruta_pred)
        else:
            df_pred = pd.DataFrame(columns=["fecha", "precio", "decision"])

        nueva_pred = {
            "fecha": ultimo.name.strftime("%Y-%m-%d"),
            "precio": precio_actual,
            "decision": decision
        }
        df_pred = pd.concat([df_pred, pd.DataFrame([nueva_pred])], ignore_index=True)
        df_pred.to_csv(ruta_pred, index=False)

        # Autoevaluación simple
        ajuste_msg = ""
        if len(df_pred) > 5:
            ultimos = df_pred.tail(5).reset_index(drop=True)
            fallos = 0
            for i in range(len(ultimos) - 1):
                d1 = ultimos.iloc[i]
                d2 = ultimos.iloc[i + 1]
                if d1["decision"] == "Comprar" and d2["precio"] < d1["precio"]:
                    fallos += 1
                elif d1["decision"] == "Vender" and d2["precio"] > d1["precio"]:
                    fallos += 1
            if fallos >= 3:
                rsi_buy = max(10, rsi_buy - 2)
                rsi_sell = min(90, rsi_sell + 2)
                ajuste_msg = f"\n⚙️ Ajustando parámetros RSI -> Buy: {rsi_buy}, Sell: {rsi_sell}"

        # Gráfico
        plt.figure(figsize=(10, 4))
        plt.plot(data.index, data["Close"], label="Precio cierre", color="blue")
        plt.title(f"Histórico de precios - {nombre}")
        plt.xlabel("Fecha")
        plt.ylabel("Precio (USD)")
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        grafico_path = "grafico.png"
        plt.savefig(grafico_path)
        plt.close()

        resultado = (
            f"📈 Criptomoneda: {nombre} ({ticker})\n"
            f"📅 Fecha: {ultimo.name.strftime('%Y-%m-%d')}\n"
            f"💰 Precio actual: {precio_actual:.2f} USD\n"
            f"📉 RSI: {rsi_actual:.2f}  (Buy < {rsi_buy}, Sell > {rsi_sell})\n"
            f"📊 Tendencia: {tendencia_actual}\n"
            f"📌 Recomendación: {decision}\n"
            f"📁 Historial guardado en: {ruta_csv}\n"
            f"📁 Predicción guardada en: {ruta_pred}"
            f"\n\n⏳ Esperando 5 minutos hasta el próximo cálculo... Revisando RSI y MACD...{ajuste_msg}"
        )

        return resultado, grafico_path

    except Exception as e:
        return f"❌ Error: {str(e)}", None

# Interfaz Gradio
interfaz = gr.Interface(
    fn=analizar_gradio,
    inputs=gr.Dropdown(choices=list(criptos.keys()), label="Selecciona una criptomoneda"),
    outputs=[
        gr.Textbox(label="Resultado del análisis"),
        gr.Image(type="filepath", label="Gráfico de precios")
    ],
    title="📊 Análisis Técnico de Criptomonedas con Historial y Autoajuste",
    description="Calcula RSI, MACD y genera recomendaciones. Guarda datos y ajusta su propio criterio con el tiempo."
)

interfaz.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.9 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8610b4f3faff8416d7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
